In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, SimpleRNN, LSTM, GRU
from tensorflow.keras import utils
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print(tf.__version__)

2.1.0


In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [62]:
# Построчное чтение
with open("Robinzon.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("utf-8", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

In [63]:
# Создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [64]:
# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [65]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [75]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 32, 256
NUM_ITERATIONS = 22 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 2
NUM_PREDS_PER_EPOCH = 200

In [76]:
'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой.
За ним последует полносвязный слой связанный с набором возможных следующих символов, 
которые конвертированы в вероятностные результаты через стандартную softmax активацию 
с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''
model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")  #  rmsprop

In [77]:
# Выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):
    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1
        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]
        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")
        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Train on 15839 samples
Epoch 1/2
15839/15839 [==============================] - 2s 134us/sample - loss: 3.1105
Epoch 2/2
15839/15839 [==============================] - 1s 90us/sample - loss: 2.6141
Генерация из посева: устынном б
устынном бо поробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ко оробил ==================================================
Итерация #: 1
Train on 15839 samples
Epoch 1/2
15839/15839 [==============================] - 1s 91us/sample - loss: 2.3775
Epoch 2/2
15839/15839 [==============================] - 1s 90us/sample - loss: 2.2384
Генерация из посева: то не попа
то не попалить в порего поробля на поробля на поробля на поробля на поробля на поробля на поробля на поробля на поробля на поробля на поробля на поробля на поробля на поробля на поробля на поробля на поробля на==========================================

15839/15839 [==============================] - 1s 93us/sample - loss: 0.1261
Epoch 2/2
15839/15839 [==============================] - 1s 91us/sample - loss: 0.1262
Генерация из посева: ой корабля
ой корабля. Мужчины за месяц сделали пирогу из ствола кедрового дерева. Ему удалось спастись. Робинзон посетил корабль дал течь. Проходившее мимо судно бросило им шлюпку, и к вечеру им удалось спастись. Робинзо==================================================
Итерация #: 17
Train on 15839 samples
Epoch 1/2
15839/15839 [==============================] - 1s 91us/sample - loss: 0.1222
Epoch 2/2
15839/15839 [==============================] - 1s 91us/sample - loss: 0.1186
Генерация из посева: час – 1 се
час – 1 сентября 1651 года Робинзон понилает, что их корабль попал в струю морского течения, уносившего его все дальше. Однако вскоре течение ослабело и Робинзону было плохо и страшно от сильной качки. Он тысяч==================================================
Итерация #: 18
Train on 15839 samples


#### 17. Робинзон понимает, что их корабль попал в струю морского течения, уносившего его все дальше. Однако вскоре течение ослабело и Робинзону было плохо и страшно от сильной качки.